![](2024-08-14-18-07-11.png)

# Detection Database Merge
## Author: Diversa
## Last Update: 27/8/2024
## Proyect: Feminist Urban Sense
## Contact: hello@diversa.studio
---

### 1.- Enviroment Preparation

In [ ]:
%pip install numpy
%pip install matplotlib
%pip install os
%pip install shutil
%pip install pandas

In [ ]:
import os
import pandas as pd
import re

### 2.- Column Creation

In [ ]:

# Function to extract country and category from the file path
def extract_country_and_category(file_path):
    path_parts = file_path.split(os.sep)
    country = path_parts[-2].capitalize()  # Capitalize the first letter of the country name
    category = path_parts[-1].replace('.csv', '')  # The category is the file name without extension
    return country, category

# Function to process a CSV file
def process_csv(file_path):
    # Load the CSV
    df = pd.read_csv(file_path)
    
    # Extract country and category from the file path
    country, category = extract_country_and_category(file_path)
    
    # Add new columns
    df['country'] = country
    df['category'] = category
    
    # Create the output directory if it doesn't exist
    output_dir = os.path.join(os.path.dirname(file_path), 'iteration1')
    os.makedirs(output_dir, exist_ok=True)
    
    # Save the modified CSV to the new directory
    output_file = os.path.join(output_dir, os.path.basename(file_path))
    df.to_csv(output_file, index=False)
    print(f'Saved: {output_file}')
# Directory where your CSVs are stored
root_dir = r'your_path'

# Iterate through all CSV files in the directory tree
for subdir, _, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(subdir, file)
            process_csv(file_path)


### 3.- Pivot and Column Filling

In [ ]:


# Function to process each CSV file
def process_csv(file_path):
    # Load the CSV
    df = pd.read_csv(file_path)
    
    # Add columns for angles
    angles = [0, 90, 180, 270]
    for angle in angles:
        df[str(angle)] = None
    
    # Extract the image number and angle, and fill the corresponding angle column
    for i, row in df.iterrows():
        try:
            # Extract the image number
            image_number = re.search(r'streetview_(\d+)_', row['imagen']).group(1)
            df.at[i, 'image'] = image_number
            
            # Extract the angle
            angle = re.search(r'_(\d+)_out', row['imagen']).group(1)
            df.at[i, angle] = row['detected']
        
        except AttributeError:
            print(f"Skipping row {i}: 'imagen' column does not match expected pattern.")
            continue
    
    # Remove the original 'imagen' and 'detected' columns
    df.drop(columns=['imagen', 'detected'], inplace=True)
    
    # Pivot the table to have one row per image
    df_pivoted = df.groupby(['image', 'country', 'category']).first().reset_index()
    
    return df_pivoted

# Directory where your CSVs are stored
root_dir = r'your_path'

# Iterate through all CSV files in the directory tree
for subdir, _, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(subdir, file)
            
            # Process the CSV
            df_pivoted = process_csv(file_path)
            
            # Create the output directory if it doesn't exist
            output_dir = os.path.join(subdir, 'iteration2')
            os.makedirs(output_dir, exist_ok=True)
            
            # Save the modified CSV to the new directory
            output_file = os.path.join(output_dir, os.path.basename(file_path))
            df_pivoted.to_csv(output_file, index=False)
            print(f'Saved: {output_file}')

### 4.- Inner Join Between Category

In [ ]:
# Load the streetview CSV (common to all)
streetview_file_path ='path/to/streetview_bdd.csv'
df_streetview = pd.read_csv(streetview_file_path)

# Convert the 'country' column in the streetview DataFrame to lowercase
df_streetview['country'] = df_streetview['country'].str.lower()

# Define the path for the specific country's category CSV
category_file_path =  'path/to/argentina/iteration1/iteration2/autonomy.csv'  # Replace with the actual path

# Load the specific category CSV
df_category = pd.read_csv(category_file_path)

# Convert the 'country' column in the category DataFrame to lowercase
df_category['country'] = df_category['country'].str.lower()

# Perform the inner join on 'image' and 'country'
df_joined = pd.merge(df_streetview, df_category, how='inner', 
                     left_on=['numero_img', 'country'], 
                     right_on=['image', 'country'])

# Define the output directory where you want to save the final combined CSV
output_dir = 'path/to/output/directory'  # Replace with your desired directory

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define the output file name
output_file_name = 'argentina_autonomy_joined.csv'  # Name the file as you wish

# Combine the directory and file name to create the full output file path
output_file_path = os.path.join(output_dir, output_file_name)

# Save the joined DataFrame to the new CSV file
df_joined.to_csv(output_file_path, index=False)

print(f"Inner join completed and saved to {output_file_path}")

In [ ]:

# Load the streetview CSV (common to all)
streetview_file_path = 'path/to/streetview_bdd.csv'
df_streetview = pd.read_csv(streetview_file_path)

# Convert the 'country' column in the streetview DataFrame to lowercase
df_streetview['country'] = df_streetview['country'].str.lower()

# Define the list of paths for each country's category CSV
category_file_paths = [
    'path/to/argentina/iteration1/iteration2/representativity.csv',
    'path/to/brasil/iteration1/iteration2/representativity.csv',
    'path/to/peru/iteration1/iteration2/representativity.csv',
    'path/to/tailandia/iteration1/iteration2/representativity.csv',
    'path/to/vietnam/iteration1/iteration2/representativity.csv'
    # Add more paths for other countries
]

# Initialize an empty DataFrame to hold the combined results
combined_df = pd.DataFrame()

# Iterate over each category file path
for category_path in category_file_paths:
    # Load the category CSV
    df_category = pd.read_csv(category_path)
    
    # Convert the 'country' column in the category DataFrame to lowercase
    df_category['country'] = df_category['country'].str.lower()
    
    # Perform the inner join on 'image' and 'country'
    df_joined = pd.merge(df_streetview, df_category, how='inner', 
                         left_on=['numero_img', 'country'], 
                         right_on=['image', 'country'])
    
    # Append the result to the combined DataFrame
    combined_df = pd.concat([combined_df, df_joined], ignore_index=True)

# Define the output directory where you want to save the final combined CSV
output_dir = 'path/to/output/directory'  # Replace with your desired directory

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define the output file name
output_file_name = 'proximity_joined_result_combined.csv'

# Combine the directory and file name to create the full output file path
output_file_path = os.path.join(output_dir, output_file_name)

# Save the combined DataFrame to the new CSV file
combined_df.to_csv(output_file_path, index=False)

print(f"All inner joins completed and saved to {output_file_path}")


### 5.- Final Merge

In [ ]:

# List of CSV file paths that you want to modify
csv_file_paths = [
    'path/to/autonomy.csv',
    'path/to/diversity.csv',
    'path/to/proximity.csv',
    'path/to/representativity.csv',
    'path/to/safety.csv',
    'path/to/vitality.csv'
]

# Iterate over each CSV file
for file_path in csv_file_paths:
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue
    
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Check if the 'country' column exists
    if 'country' in df.columns:
        # Replace 'ec_co' with 'ecuador_colombia' in the 'country' column
        df['country'] = df['country'].replace('Ec_co', 'ecuador_colombia')
        
        # Overwrite the original CSV file with the updated DataFrame
        df.to_csv(file_path, index=False)
        
        print(f"Updated and saved: {file_path}")
    else:
        print(f"'country' column not found in {file_path}")



In [ ]:

# Load the streetview CSV (common to all)
streetview_file_path = 'path/to/streetview_bdd.csv'
df_streetview = pd.read_csv(streetview_file_path)

# Convert the 'country' column in the streetview DataFrame to lowercase
df_streetview['country'] = df_streetview['country'].str.lower()

# Define lists of paths for each category's CSVs
autonomy_file_paths = [
    'path/to/autonomy.csv',
    'path/to/autonomy.csv',
    'path/to/autonomy.csv',
    'path/to/autonomy.csv',
    'path/to/autonomy.csv',
    'path/to/autonomy.csv'
]

diversity_file_paths = [
    'path/to/diversity.csv',
    'path/to/diversity.csv',
    'path/to/diversity.csv',
    'path/to/diversity.csv',
    'path/to/diversity.csv',
    'path/to/diversity.csv'
]

proximity_file_paths = [
    'path/to/proximity.csv',
    'path/to/proximity.csv',
    'path/to/proximity.csv',
    'path/to/proximity.csv',
    'path/to/proximity.csv',
    'path/to/proximity.csv'
]

representativity_file_paths = [
    'path/to/representativity.csv',
    'path/to/representativity.csv',
    'path/to/representativity.csv',
    'path/to/representativity.csv',
    'path/to/representativity.csv'
]

safety_file_paths = [
    'path/to/safety.csv',
    'path/to/safety.csv',
    'path/to/safety.csv',
    'path/to/safety.csv',
    'path/to/safety.csv',
    'path/to/safety.csv'
]

vitality_file_paths = [
    'path/to/vitality.csv',
    'path/to/vitality.csv',
    'path/to/vitality.csv',
    'path/to/vitality.csv',
    'path/to/vitality.csv',
    'path/to/vitality.csv'
]

# Combine all category file lists into a dictionary with category names as keys
category_file_lists = {
    'autonomy': autonomy_file_paths,
    'diversity': diversity_file_paths,
    'proximity': proximity_file_paths,
    'representativity': representativity_file_paths,
    'safety': safety_file_paths,
    'vitality': vitality_file_paths,
}

# Process each category
for category_name, category_file_paths in category_file_lists.items():
    # Initialize an empty DataFrame to hold the combined results for the category
    combined_df = pd.DataFrame()
    
    # Iterate over each file path for the current category
    for category_path in category_file_paths:
        # Load the category CSV
        df_category = pd.read_csv(category_path)
        
        # Convert the 'country' column in the category DataFrame to lowercase
        df_category['country'] = df_category['country'].str.lower()
        
        # Perform the inner join on 'image' and 'country'
        df_joined = pd.merge(df_streetview, df_category, how='inner', 
                             left_on=['numero_img', 'country'], 
                             right_on=['image', 'country'])
        
        # Append the result to the combined DataFrame
        combined_df = pd.concat([combined_df, df_joined], ignore_index=True)
    
    # Define the output directory where you want to save the final combined CSV
    output_dir = 'path/to/output/directory'  # Replace with your desired directory
    
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Define the output file name based on the category name
    output_file_name = f'{category_name}_final.csv'  # e.g., 'autonomy_final.csv'
    
    # Combine the directory and file name to create the full output file path
    output_file_path = os.path.join(output_dir, output_file_name)
    
    # Save the combined DataFrame to the new CSV file
    df_joined.to_csv(output_file_path, index=False)
    
    print(f"All inner joins for {category_name} completed and saved to {output_file_path}")

